In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [2]:
# Load the Excel file
file_name = "SES_2024_tidy.xlsx.coredownload.xlsx"
sheet_name = "T3.5"

# Read the data from the Excel file
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df.fillna(0)

# Treat the random 's' as 0
df['Kwh_Per_Acc_Cleaned'] = df['Kwh Per Acc'].apply(lambda x: 0 if isinstance(x, str) and 's' in x else x) 

# Filter the data for the year 2024 and 'Dwelling Type' as 'overall'
filtered_data = df[(df['Year'] == 2024) & (df['Dwelling Type'] == 'Overall') & (df['Month'] == 'Annual')]

# Calculate total consumption per region
regions = ['Central Region', 'East Region', 'North East Region', 'North Region', 'West Region']
demand_kW_per_region = {
    region: filtered_data[filtered_data['Region'] == region]['Kwh_Per_Acc_Cleaned'].astype(float).sum()
    for region in regions
}

demand_kW_per_region

{'Central Region': 12698.599999999999,
 'East Region': 3884.1000000000004,
 'North East Region': 3428.7000000000003,
 'North Region': 3354.5,
 'West Region': 2853.1000000000004}

In [3]:
df_carpark = pd.read_csv('Checkmark2.csv')

# Calculate the number of car parks per region
carpark_per_region = {
    region: df_carpark[df_carpark['region'] == region].shape[0]
    for region in regions
}

# Output the result
carpark_per_region

{'Central Region': 541,
 'East Region': 400,
 'North East Region': 493,
 'North Region': 303,
 'West Region': 506}

In [4]:
# Sort the DataFrame by region to ensure consistent labeling
df_carpark.sort_values(by='region', inplace=True)

# Initialize a new column for carpark labels
df_carpark['carpark_label'] = 0

# Group the DataFrame by the 'region' column
grouped = df_carpark.groupby('region')

# Assign labels to each carpark within each region
for region, group in grouped:
    # Assign labels from 1 to the number of carparks in the region
    df_carpark.loc[group.index, 'carpark_label'] = range(1, len(group) + 1)
    

In [5]:
# Define other parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}
var_cost = {'Monocrystalline': 266.125, 'Polycrystalline': 265.85, 'Thin-film': 265.5}
fixed_cost = 39457.75
available_space_m2 = 2559.5
budget_limit = 40_000_000 # Budget limit in dollars

# Calculate the total area by region constraint
area_by_region_constraint = {
    region: carpark_per_region[region] * available_space_m2
    for region in carpark_per_region
}

area_by_region_constraint

{'Central Region': 1384689.5,
 'East Region': 1023800.0,
 'North East Region': 1261833.5,
 'North Region': 775528.5,
 'West Region': 1295107.0}

In [6]:
# Define the range of carpark sizes
min_size = 1808
max_size = 3311

# Calculate the mean and standard deviation
mean_size = available_space_m2
std_dev = (max_size - min_size) / 6  # Assuming 99.7% of data falls within the range

# Generate carpark sizes
num_carparks = len(df_carpark)  # Number of carparks in the CSV
carpark_sizes = np.random.normal(loc=mean_size, scale=std_dev, size=num_carparks)

# Clip the values to ensure they fall within the specified range
carpark_sizes = np.clip(carpark_sizes, min_size, max_size)

# Add the generated sizes as a new column in the DataFrame
df_carpark['carpark_area'] = carpark_sizes

# Save the updated DataFrame to a new CSV file
df_carpark.to_csv('Checkmark4.csv', index=False)

carpark_areas = { (row['region'], row['carpark_label']): row['carpark_area'] for _, row in df_carpark.iterrows() }

In [7]:
# Create a new model
model = gp.Model("solar_optimization")

x = model.addVars(
    [(region, panel_type, carpark) 
     for region in regions 
     for panel_type in types_of_panels 
     for carpark in range(1, carpark_per_region[region] + 1)],
    vtype=GRB.CONTINUOUS, 
    name="x"
)

y = model.addVars(
    [(region, carpark) 
     for region in regions 
     for carpark in range(1, carpark_per_region[region] + 1)],
    vtype=GRB.BINARY, 
    name="y"
)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22


In [8]:
model.setObjective( gp.quicksum( fixed_cost * y[region, carpark] + gp.quicksum(var_cost[panel] * x[region, panel, carpark] for panel in types_of_panels) for region in regions for carpark in range(1, carpark_per_region[region] + 1) ), GRB.MINIMIZE )

<h3>Can budget by maximum number of panels you want to build if you do this way it's 1000 in the last line</h3>

In [9]:
# # Constraints
# for i in regions:
#     # Demand constraint with maximum power per panel
#     model.addConstr(
#         gp.quicksum(efficiency[j] * max_power_per_panel_kW * x[i, j] for j in types_of_panels) >= demand_kW_per_region[i],
#         name=f"demand_{i}"
#     )

#     # Space constraint
#     model.addConstr(
#         gp.quicksum(x[i, j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
#         name=f"space_{i}"
#     )

#     # Installation constraints
#     for j in types_of_panels:
#         model.addConstr(x[i, j] <= 1000 * y[i, j], name=f"install_{i}_{j}")

<h3>Or if you do this way then you budget by the money set in the cell above. For now I put $1 mil</h3>

In [10]:
# Add constraints
for region in regions:
    for carpark in range(1, carpark_per_region[region] + 1):
        carpark_key = (region, carpark)
        if carpark_key in carpark_areas:
            # Space constraint for each carpark using the carpark_area from the CSV
            model.addConstr(
                gp.quicksum(10 * x[region, panel, carpark] for panel in types_of_panels) <= carpark_areas[carpark_key],
                f"Space_Constraint_{region}_{carpark}"
            )
        else:
            raise ValueError(f"Carpark area not found for {carpark_key}")

    # Demand constraint for each region
    model.addConstr(
        gp.quicksum(efficiency[panel] * gp.quicksum(x[region, panel, carpark] for carpark in range(1, carpark_per_region[region] + 1)) for panel in types_of_panels) >= demand_kW_per_region[region],
        f"Demand_Constraint_{region}"
    )
    
    # Budget constraint for each region
    model.addConstr(
        gp.quicksum(fixed_cost * y[region, carpark] + gp.quicksum(var_cost[panel] * x[region, panel, carpark] for panel in types_of_panels) for carpark in range(1, carpark_per_region[region] + 1)) <= budget_limit,
        f"Budget_Constraint_{region}"
    )
    
    # Big M and non-negativity constraints
    for panel in types_of_panels:
        for carpark in range(1, carpark_per_region[region] + 1):
            model.addConstr(x[region, panel, carpark] <= GRB.INFINITY * y[region, carpark], f"Big_M_Constraint_{region}_{panel}_{carpark}")
            model.addConstr(x[region, panel, carpark] >= 0, f"Non-Negativity_{region}_{panel}_{carpark}")

In [ ]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 15711 rows, 8972 columns and 42617 nonzeros
Model fingerprint: 0x291ca902
Variable types: 6729 continuous, 2243 integer (2243 binary)
Coefficient statistics:
  Matrix range     [7e-02, 1e+100]
  Objective range  [3e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 4e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 13459 rows and 0 columns
Presolve time: 0.66s
Presolved: 2252 rows, 8972 columns, 23461 nonzeros
Variable types: 6729 continuous, 2243 integer (2243 binary)

Root relaxation: objective 7.096176e+07, 2314 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

In [ ]:
# Check if the model has an optimal solution
if model.status == GRB.OPTIMAL:
    # Create lists to store the results
    results = []

    # Iterate over regions and carparks to collect results
    for region in regions:
        for carpark in range(1, carpark_per_region[region] + 1):
            # Collect the binary decision variable y
            y_value = y[region, carpark].X
            # Collect the continuous decision variables x for each panel type
            x_values = {panel: x[region, panel, carpark].X for panel in types_of_panels}
            # Append the results to the list
            results.append({
                'Region': region,
                'Carpark': carpark,
                'y': y_value,
                **x_values
            })

    # Create a DataFrame from the results
    df_results = pd.DataFrame(results)

    # Print the DataFrame
    print(df_results)
else:
    print("No optimal solution found.")

df_results.to_excel("results.xlsx")

              Region  Carpark    y  Monocrystalline  Polycrystalline  \
0     Central Region        1  0.0             0.00              0.0   
1     Central Region        2  0.0             0.00              0.0   
2     Central Region        3  0.0             0.00              0.0   
3     Central Region        4  1.0           255.95              0.0   
4     Central Region        5  0.0             0.00              0.0   
...              ...      ...  ...              ...              ...   
2238     West Region      502  0.0             0.00              0.0   
2239     West Region      503  0.0             0.00              0.0   
2240     West Region      504  0.0             0.00              0.0   
2241     West Region      505  0.0             0.00              0.0   
2242     West Region      506  0.0             0.00              0.0   

      Thin-film  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  
...         ...  
2238     

In [ ]:
# Set the txt file to see the logs
model.setParam('LogFile', 'gurobi_log.txt')
model.setParam('OutputFlag', 1) 

# Optimize the model
model.optimize()

# In case we mess up and we wanna see the variables 
if model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")

Set parameter LogFile to value "gurobi_log.txt"
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 45 rows, 20 columns and 95 nonzeros
Model fingerprint: 0x49401357
Variable types: 15 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [7e-02, 1e+100]
  Objective range  [3e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 4e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 2 rows, 3 columns, 6 nonzeros

Continuing optimization...


Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 4.67142e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.6714164583

In [ ]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in regions:
        for j in types_of_panels:
            print(f"Region: {i}, {j}: {x[i, j].x} kwp System, Build: {y[i].x}")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found:
Region: Central Region, Monocrystalline: 84657.33333333333 kwp System, Build: 1.0
Region: Central Region, Polycrystalline: 0.0 kwp System, Build: 1.0
Region: Central Region, Thin-film: 0.0 kwp System, Build: 1.0
Region: East Region, Monocrystalline: 25894.0 kwp System, Build: 1.0
Region: East Region, Polycrystalline: 0.0 kwp System, Build: 1.0
Region: East Region, Thin-film: 0.0 kwp System, Build: 1.0
Region: North East Region, Monocrystalline: 22858.0 kwp System, Build: 1.0
Region: North East Region, Polycrystalline: 0.0 kwp System, Build: 1.0
Region: North East Region, Thin-film: 0.0 kwp System, Build: 1.0
Region: North Region, Monocrystalline: 22363.333333333332 kwp System, Build: 1.0
Region: North Region, Polycrystalline: 0.0 kwp System, Build: 1.0
Region: North Region, Thin-film: 0.0 kwp System, Build: 1.0
Region: West Region, Monocrystalline: 19020.666666666668 kwp System, Build: 1.0
Region: West Region, Polycrystalline: 0.0 kwp System, Build: 1.0
Region: 

In [ ]:
for constr in model.getConstrs():
    print(f"Constraint: {constr.ConstrName}, Expression: {model.getRow(constr)}, RHS: {constr.RHS}")

Constraint: Demand_Constraint_Central Region, Expression: 0.15 x[Central Region,Monocrystalline] + 0.13 x[Central Region,Polycrystalline] + 0.07 x[Central Region,Thin-film], RHS: 12698.599999999999
Constraint: Space_Constraint_Central Region, Expression: 10.0 x[Central Region,Monocrystalline] + 10.0 x[Central Region,Polycrystalline] + 10.0 x[Central Region,Thin-film], RHS: 1384689.5
Constraint: Budget_Constraint_Central Region, Expression: 266.125 x[Central Region,Monocrystalline] + 265.85 x[Central Region,Polycrystalline] + 265.5 x[Central Region,Thin-film] + 39457.75 y[Central Region], RHS: 40000000.0
Constraint: Big_M_Constraint_Central Region_Monocrystalline, Expression: x[Central Region,Monocrystalline] + -1e+100 y[Central Region], RHS: 0.0
Constraint: Non-Negativity_Central Region_Monocrystalline, Expression: x[Central Region,Monocrystalline], RHS: 0.0
Constraint: Big_M_Constraint_Central Region_Polycrystalline, Expression: x[Central Region,Polycrystalline] + -1e+100 y[Central Re

In [ ]:
# Check if the model is infeasible
if model.status == GRB.INFEASIBLE:
    # Compute the IIS
    model.computeIIS()
    
    # Print the constraints in the IIS
    print("Constraints in the IIS:")
    for constr in model.getConstrs():
        if constr.IISConstr:
            print(constr.ConstrName)
    
    # Print the variable bounds in the IIS
    print("Variable bounds in the IIS:")
    for var in model.getVars():
        if var.IISLB:
            print(f"{var.VarName} (lower bound)")
        if var.IISUB:
            print(f"{var.VarName} (upper bound)")
else:
    print("Optimal solution found.")

Optimal solution found.
